# Installation of the modules

In [16]:
!pip install -r requirement.txt
!pip install torchinfo

Defaulting to user installation because normal site-packages is not writeable
  Using cached cctx-1.0.0-py2.py3-none-any.whl.metadata (3.4 kB)
  Using cached dotenv-0.9.9-py2.py3-none-any.whl.metadata (279 bytes)


ERROR: Could not find a version that satisfies the requirement pmlfinance (from versions: none)

[notice] A new release of pip is available: 25.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for pmlfinance


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Modules importations

In [ ]:
import os
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms
from torchvision import models

import seaborn as sns
import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Using device: cuda



[notice] A new release of pip is available: 25.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Use of torchvision to resize and normalize images

In [ ]:
transform = transforms.Compose([
    transforms.Pad(10, fill=(0,0,0)),  
    transforms.Resize((224,224)),      
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

## Load data

In [6]:
train_data = datasets.ImageFolder('dataset/train', transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

val_data = datasets.ImageFolder('dataset/val', transform=transform)
val_loader = DataLoader(val_data, batch_size=32, shuffle=True)


## Define the model

We will use a CNN that is called ResNet and he will be pre-trained.

In [7]:
num_classes = len(train_data.classes)

model = models.resnet18(pretrained=True)
# extract the number of features in the last layer
num_features = model.fc.in_features
# replace the last layer with a new layer that has the same number of inputs but num_classes outputs
model.fc = nn.Linear(num_features, num_classes)
# Move the model to the appropriate device (GPU or CPU)
model = model.to(device)


C:\Users\dylan\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\dylan\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Train the model

In [8]:
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 1: 100%|██████████| 5/5 [00:02<00:00,  2.39it/s]


Epoch 1, Loss: 1.2447


Epoch 2: 100%|██████████| 5/5 [00:00<00:00, 15.04it/s]


Epoch 2, Loss: 0.3217


Epoch 3: 100%|██████████| 5/5 [00:00<00:00, 15.27it/s]


...

Epoch 8, Loss: 0.0052


Epoch 9: 100%|██████████| 5/5 [00:00<00:00, 14.99it/s]


Epoch 9, Loss: 0.0054


Epoch 10: 100%|██████████| 5/5 [00:00<00:00, 14.90it/s]

Epoch 10, Loss: 0.0021


## Evaluate the model

In [9]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100*correct/total:.2f}%')


Accuracy: 90.48%


## Summary of the model

In [20]:
summary(model, input_size=(32, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [32, 7]                   --
├─Conv2d: 1-1                            [32, 64, 112, 112]        9,408
├─BatchNorm2d: 1-2                       [32, 64, 112, 112]        128
├─ReLU: 1-3                              [32, 64, 112, 112]        --
├─MaxPool2d: 1-4                         [32, 64, 56, 56]          --
├─Sequential: 1-5                        [32, 64, 56, 56]          --
│    └─BasicBlock: 2-1                   [32, 64, 56, 56]          --
│    │    └─Conv2d: 3-1                  [32, 64, 56, 56]          36,864
│    │    └─BatchNorm2d: 3-2             [32, 64, 56, 56]          128
│    │    └─ReLU: 3-3                    [32, 64, 56, 56]          --
│    │    └─Conv2d: 3-4                  [32, 64, 56, 56]          36,864
│    │    └─BatchNorm2d: 3-5             [32, 64, 56, 56]          128
│    │    └─ReLU: 3-6                    [32, 64, 56, 56]          --
│